In [1]:
from keras.models import Sequential,load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation
from keras.layers import Dropout, LeakyReLU       
from keras.activations import relu, softmax, tanh, hard_sigmoid

import pydot, graphviz
from keras.utils.vis_utils import plot_model

import matplotlib.pyplot as plt
import cv2
import numpy as np
import os, glob
from PIL import Image
import h5py
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
import keras
from keras.callbacks import ModelCheckpoint

from keras.initializers import he_normal
from keras import regularizers

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
from tqdm import tqdm
#import xgboost as xgb
import tensorflow as tf
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Flatten, Input
import scipy
from sklearn.metrics import fbeta_score

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np
from keras.utils.data_utils import get_file

In [5]:
densenet = keras.applications.densenet.DenseNet201(include_top=False, weights=None, input_shape=(224,224,3), pooling='max')
densenet.load_weights('densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [4]:
print(2)

2


In [19]:
vgg19 = keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [3]:
resnet = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3), pooling='max')

In [3]:
for i, layers in enumerate(resnet.layers):
    print(i, layers.name)

0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 activatio

In [6]:
def hist_equ(img_path = '', show = False, img = None):
    if img is None:
        img = cv2.imread(data_path + img_path)

    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV) # RGB to YUB color space

    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0]) # histogram equalization in Y channel

    equ = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR) # inverse transform
    
    if show:    
        comp = np.hstack((img,equ)) # horizontal stack
        cv2.imwrite('original.bmp',img)
        cv2.imwrite('hist_equ.bmp',equ)
        cv2.imwrite('comp.bmp',comp)
        plt.imshow(comp)
        plt.show()
    return equ


def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [7]:
def generate_segmented_images(_class,_id,data_split,flag_show):

    _class = str(_class)
    _id = str(_id)
    
    imgpath = 'PH2 Dataset images/'+'/IMD'+_id+'/IMD'+_id+'_Dermoscopic_Image/IMD'+_id+'.bmp'  
    maskpath = 'PH2 Dataset images/'+'/IMD'+_id+'/IMD'+_id+'_lesion/IMD'+_id+'_lesion.bmp'
    img = cv2.imread(imgpath)
    #img = hist_equ(img = img2) # not using any pre-processing
    #img = adjust_gamma(img, 1.55)
    msk = cv2.imread(maskpath)
    
    #cv2.imshow('Main Image',img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    if flag_show:
        plt.imshow(img)
        plt.show()
        #print(img.shape)

        plt.imshow(msk)
        plt.show()


    msk = msk/255 # normalizing
    max_channels = np.amax([np.amax(msk[:,:,0]), np.amax(msk[:,:,1]), np.amax(msk[:,:,2])])
    
    msk = np.ndarray.astype(msk, dtype='uint8')
    
    gen_img = img*msk
    dirc = 'F:/Research important/Thesis/Active Phase/Part1 Scin Cancer/PH2Dataset experiments/classification/' + data_split
    dirc = os.path.join(dirc,_class)
    if not os.path.exists(dirc):
        os.makedirs(dirc)
        
    gen_img_name =  dirc + '/' + _id + '.png'
    cv2.imwrite(gen_img_name,gen_img)
    if flag_show:
        print(max_channels)
        plt.imshow(gen_img)
        plt.show()



In [8]:
import pandas
df = pandas.read_excel('PH2_dataset.xlsx') # The excel file was modified, unnecessary info was removed
#print the column names
print (df.columns)
#get the values for a given column
values = df['Image Name'].values
#get a data frame with selected columns
FORMAT = ['Image Name', 'Common Nevus', 'Atypical Nevus', 'Melanoma']
df_selected = df[FORMAT]
print(df_selected.loc[0][0])
print(df_selected.loc[0][1])
print(type(df_selected.loc[0][0]))
df_selected[1:4]

Index(['Image Name', 'Histological Diagnosis', 'Common Nevus',
       'Atypical Nevus', 'Melanoma', 'Asymmetry\n(0/1/2)',
       'Pigment Network\n(AT/T)', 'Dots/Globules\n(A/AT/T)', 'Streaks\n(A/P)',
       'Regression Areas\n(A/P)', 'Blue-Whitish Veil\n(A/P)', 'White', 'Red',
       'Light-Brown', 'Dark-Brown', 'Blue-Gray', 'Black'],
      dtype='object')
IMD003
X
<class 'str'>


,Image Name,Common Nevus,Atypical Nevus,Melanoma
1,IMD009,X,NaN,NaN
2,IMD016,X,NaN,NaN
3,IMD022,X,NaN,NaN


In [10]:
num_samples = 200
class0 = []
class1 = []
class2 = []
for ns in range(num_samples):
    img_id = df_selected.loc[ns][0][-3:]
    com_nev = df_selected.loc[ns][1]
    atyp_nev = df_selected.loc[ns][2]
    melan = df_selected.loc[ns][3]
    _class = -1
    if com_nev == 'X':
        _class = 0
        class0.append(img_id)
    elif atyp_nev == 'X':
        _class = 1
        class1.append(img_id)
    elif melan == 'X':
        _class = 2
        class2.append(img_id)
    #print(_class)
    #generate_segmented_images(_class,img_id,flag_show = False)
    

In [15]:
import random
random.seed(0) # to reproduce results

train_split = 0.80
random.shuffle(class0)
random.shuffle(class1)
random.shuffle(class2)

train_0 = class0[0:int(len(class0)*train_split)]
test_0 = class0[int(len(class0)*train_split):]
print(train_0)
print(test_0)

train_1 = class1[0:int(len(class1)*train_split)]
test_1 = class1[int(len(class1)*train_split):]
print(train_1)
print(test_1)

train_2 = class2[0:int(len(class2)*train_split)]
test_2 = class2[int(len(class2)*train_split):]
print(train_2)
print(test_2)

['134', '394', '159', '390', '161', '177', '039', '017', '372', '397', '022', '041', '101', '150', '374', '380', '147', '050', '132', '395', '371', '378', '045', '207', '112', '399', '016', '010', '175', '199', '118', '197', '092', '003', '025', '383', '375', '208', '144', '176', '364', '392', '198', '182', '400', '365', '024', '152', '042', '173', '125', '135', '143', '103', '035', '385', '206', '196', '402', '160', '162', '146', '105', '381']
['009', '384', '204', '133', '379', '156', '107', '367', '389', '044', '203', '142', '200', '108', '038', '020']
['431', '032', '166', '356', '018', '430', '376', '153', '339', '031', '368', '256', '382', '169', '013', '139', '278', '036', '140', '436', '254', '279', '331', '437', '434', '006', '280', '078', '043', '433', '226', '328', '306', '138', '075', '033', '210', '398', '027', '396', '360', '171', '393', '427', '008', '347', '015', '170', '040', '243', '021', '048', '304', '386', '305', '157', '312', '432', '019', '120', '369', '370', '15

In [16]:
for i in range(len(train_0)):
    generate_segmented_images(0,train_0[i],'train',flag_show = False)

for i in range(len(test_0)):
    generate_segmented_images(0,test_0[i],'test',flag_show = False)
    
for i in range(len(train_1)):
    generate_segmented_images(1,train_1[i],'train',flag_show = False)

for i in range(len(test_1)):
    generate_segmented_images(1,test_1[i],'test',flag_show = False)
    
    
for i in range(len(train_2)):
    generate_segmented_images(2,train_2[i],'train',flag_show = False)

for i in range(len(test_2)):
    generate_segmented_images(2,test_2[i],'test',flag_show = False)

In [7]:
train_path = 'train_aug'
test_path = 'test'

random_seed = 1997
random.seed(random_seed)
np.random.seed(random_seed)

# Extract features from an arbitrary intermediate layer
from keras.models import Model
from keras.preprocessing import image
#model_fe = Model(inputs=cmod.input, outputs=cmod.get_layer('conv2d_6').output)


# use ResNet50 model extract feature from fc1 layer
base_model1 = None #vgg19 #ResNet50(weights='imagenet', pooling=max, include_top = False)
base_model2 = resnet #resnet #ResNet50(weights='imagenet', pooling=max, include_top = False)

input = Input(shape=(224,224,3),name = 'image_input')
#x = base_model1(input)
#x1 = Flatten()(x)
#model1 = Model(inputs=input, outputs=x1)

x_ = base_model2(input)
#x2 = Flatten()(x_)
model2 = Model(inputs=input, outputs=x_)

X_train_vgg19 = []
X_train_resnet = []
X_train_mod3 = []
y_train = []

for class_ in range(3):
    for fnam in glob.glob(os.path.join(train_path,str(class_),'*')):
        img = image.load_img(fnam, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        #features1 = model1.predict(x)
        #features_reduce1 =  features1.squeeze()
        #X_train_vgg19.append(features_reduce1)
        
        features2 = model2.predict(x)
        features_reduce2 =  features2.squeeze()
        X_train_resnet.append(features_reduce2)
        
        #features3 = model_fe.predict(x)
        #features_reduce3 =  features3.squeeze()
        #X_train_mod3.append(features_reduce3)

        targets = np.zeros(3)
        targets[class_] = 1
        y_train.append(targets)

#X = np.array(X_train_vgg19 + X_train_resnet)
y = np.array(y_train, np.uint8)

X_test_vgg19 = []
X_test_resnet = []
X_test_mod3 = []

y_test = []

for class_ in range(3):
    for fnam in glob.glob(os.path.join(test_path,str(class_),'*')):
        img = image.load_img(fnam, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        #features1 = model1.predict(x)
        #features_reduce1 =  features1.squeeze()
        #X_test_vgg19.append(features_reduce1)
        
        features2 = model2.predict(x)
        features_reduce2 =  features2.squeeze()
        X_test_resnet.append(features_reduce2)
        
        #features3 = model_fe.predict(x)
        #features_reduce3 =  features3.squeeze()
        #X_test_mod3.append(features_reduce3)
        
        
        y_test.append(class_)

#X_test = X_test_vgg19 + X_test_resnet

In [11]:
print(X_test_resnet[0].shape)
print(type(X_test_resnet))
print(type(X_train_resnet))

(2048,)
<class 'list'>
<class 'list'>


In [12]:
X = []
for i in range(len(X_train_resnet)):
    cx = [] 
    for j in range(len(X_train_resnet[0])):
        cx.append(X_train_resnet[i][j])#np.array(X_train_vgg19 + X_train_resnet)
    #for j in range(len(X_train_resnet[0])):
    #    cx.append(X_train_resnet[i][j])#np.array(X_train_vgg19 + X_train_resnet)
    #for j in range(len(X_train_mod3[0])):
    #    cx.append(X_train_mod3[i][j])#np.array(X_train_vgg19 + X_train_resnet)
        
    X.append(cx)

In [13]:
X_test = []
for i in range(len(X_test_resnet)):
    cx = [] 
    for j in range(len(X_test_resnet[0])):
        cx.append(X_test_resnet[i][j])#np.array(X_train_vgg19 + X_train_resnet)
    #for j in range(len(X_test_resnet[0])):
    #    cx.append(X_test_resnet[i][j])#np.array(X_train_vgg19 + X_train_resnet)
    #for j in range(len(X_test_mod3[0])):
    #    cx.append(X_test_mod3[i][j])
        
    X_test.append(cx)

In [7]:
X_train_mod3 = np.array(X_train_mod3)
print(X_train_mod3[0])

[[[-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  ...
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]]

 [[-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  ...
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  [-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]]

 [[-427.66324 -113.42633 -314.65555 ... -287.33    -256.3068  -134.88426]
  [-427.66324 -113.426

In [8]:
X_train_mod3.shape

(460, 215, 215, 32)

In [11]:
X_test_mod3 = np.array(X_test_mod3)

In [12]:
X_test_mod3.shape

(40, 215, 215, 32)

In [9]:
X_train_mod3_conv = np.reshape(X_train_mod3,(460,1479200))

In [13]:
X_test_mod3_conv = np.reshape(X_test_mod3,(40,1479200))

In [ ]:
print(2)

In [14]:
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(n_jobs=2, random_state=0,class_weight = [{0:1,1:1},
                                                                     {0:1,1:1},
                                                                     {0:1,1:2}])

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(X_train_resnet, y)

RandomForestClassifier(bootstrap=True,
            class_weight=[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 2}],
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [47]:
print(34)

34


In [39]:
y_test = [] # 16, 16, 8
for i in range(16):
    y_test.append(0)
for i in range(16):
    y_test.append(1)
for i in range(8):
    y_test.append(2)

In [15]:
pred_v = clf.predict(X_test)

In [16]:
pred = np.argmax(pred_v,axis=1)
print(pred[0])
print(y_test[0])

0
0


In [17]:
cnt = 0
tot = 0
for i in range(len(pred)):
    if(pred[i]==y_test[i]):
        cnt+=1
    #if(pred[i]==0 and y_test[i]==1):
    #    cnt+=1
    #if(pred[i]==1 and y_test[i]==0):
    #    cnt+=1
    
    tot += 1
print(cnt/tot)   # resnet 2 class - 87.5, 3 class - 62.5
                 # vgg19  2 class - 0.5,  3 class - 85
                 # feature concat 3 class -  57.5  ,  2 class - 90
                 # augmentation + feature concat 3 class 0.675, 2 class 90%
                 # augmentation + mod3 2 class 90%, 2 class .55, features extracted from last layer

0.55


In [18]:
for i in range(len(pred)):
    print(pred[i],' ',y_test[i])

0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
0   0
1   0
1   0
0   0
0   0
1   1
0   1
0   1
0   1
0   1
0   1
0   1
1   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
0   1
2   2
2   2
2   2
2   2
0   2
2   2
2   2
0   2


In [5]:
## augmentation
import Augmentor
p = Augmentor.Pipeline("train")
#p.random_distortion(probability = 1, grid_width = 10, grid_height = 10, magnitude = 10)
#p.flip_random(probability = 1)
p.rotate_without_crop(probability=1, max_left_rotation=360, max_right_rotation=360, expand=False)
p.skew(probability=1, magnitude=0.8)
p.sample(100)

Initialised with 160 image(s) found.
Output directory set to train\output.

Processing <PIL.Image.Image image mode=RGB size=766x576 at 0xFFB70E7B00>: 100%|█| 100/100 [00:08<00:00, 12.17 Samples/s]


In [4]:
cmod = load_model('best_weight3_mod.hdf5')

In [5]:
cmod.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 220, 220, 16)      1216      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 219, 219, 16)      0         
_________________________________________________________________
activation_9 (Activation)    (None, 219, 219, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 215, 215, 32)      12832     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 107, 107, 32)      0         
_________________________________________________________________
activation_10 (Activation)   (None, 107, 107, 32)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 366368)            0         
__________

In [17]:
# Extract features from an arbitrary intermediate layer
from keras.models import Model
from keras.preprocessing import image
model_fe = Model(inputs=cmod.input, outputs=cmod.get_layer('conv2d_6').output)

# load an image and preprocess it
img_path = 'test.bmp'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

# get the features 
features = model_fe.predict(x)